In [1]:
# importing all the required libraries
import pandas as pd
import numpy as np
import math
from sklearn import linear_model

import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import pickle

In [2]:
# # importing the data file
df= pd.read_csv("fifa21_validate.csv")

pd.set_option('display.max_columns', None)

In [3]:
# pickle.dump(MinMaxtransformer, open('scalar.pkl', 'wb'))
# pickle.dump(encoder, open('encoder.pkl', 'wb'))
# pickle.dump(lm, open('model.pkl', 'wb'))

In [4]:
scalar=pickle.load(open('scalar.pkl','rb'))
lm=pickle.load(open('model.pkl','rb'))
encoder=pickle.load(open('encoder.pkl','rb'))

In [5]:
def convert_currency(currency_str):
    if currency_str.endswith('K'):
        return float(currency_str[1:-1]) * 1000
    elif currency_str.endswith('M'):
        return float(currency_str[1:-1]) * 1000000
    else:
        return float(currency_str[1:])

In [6]:
def preprocess(df):
    df.columns= [col.lower() for col in df.columns]
#     df = df.drop_duplicates()
    df['value'] = df['value'].apply(convert_currency)
    df['wage'] = df['wage'].apply(convert_currency)
    df['release clause'] = df['release clause'].apply(convert_currency)
    
    df=df.drop(['team & contract'], axis=1)

    df['hits'] = pd.to_numeric(df['hits'], errors='coerce', downcast='integer')

    
    columns = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm',
           'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb','gk']


    for colmun in columns: # for itertion of columns
        df[colmun] = df[colmun].str.split('+',expand=True)[0] # Split of string
        df[colmun] = df[colmun].str.split('-',expand=True)[0]
    
    for column in columns:
        df[column] = pd.to_numeric(df[column], errors='coerce', downcast='float')
        
    df_c=df.select_dtypes(include = np.object)
    
    column_modes = df_c.mode().iloc[0] 
    df_c = df_c.fillna(column_modes)
        
    columns_to_delete = ['name','nationality','club','position','height', 'weight','joined','loan date end','contract']

    df_c.drop(columns=columns_to_delete, inplace=True)
    
    df_n=df.select_dtypes(include = np.number)
    df_n=df_n.drop(['id'], axis=1)
    column_means = df_n.mean()

# Fill NaN values with column means
    df_n.fillna(column_means, inplace=True)
    
    
    df=pd.concat([df_c,df_n], axis=1)


    #separate the features from the labels
    y = df['ova']
    X = df.drop(['ova'], axis=1)
    
    #categorical features and numerical ones are going to be treated differently
    X_num = X.select_dtypes(include = np.number)
    X_cat = X.select_dtypes(include = object)
    

    
    from sklearn.preprocessing import MinMaxScaler
    X_normalized = scalar.transform(X_num)
    X_normalized = pd.DataFrame(X_normalized,columns=X_num.columns)

    #one hot encoding is a way to turn categorical variables into multiple numerical columns
    from sklearn.preprocessing import OneHotEncoder    
    cols = encoder.get_feature_names_out(input_features=X_cat.columns)
    X_cat_encode = pd.DataFrame(encoder.transform(X_cat).toarray(),columns=cols)


    # concatinating the normalised and encoded data
    X = pd.concat([X_normalized,X_cat_encode], axis=1)  # np.concatenate()
    X


    from sklearn.metrics import mean_absolute_error, r2_score  
    # r2 score
    predictions = lm.predict(X)
    print("r2_score",r2_score(y, predictions))
   
    
    from sklearn.metrics import mean_squared_error
    # RMSE
    rmse = np.sqrt(mean_squared_error(y,predictions))
    print("rmse",rmse)

    #MAE 
    mae = mean_absolute_error(y, predictions)
    print("mae",mae)

    MSE = mean_squared_error(y,predictions)
    print("MSE",MSE.round(2))
          
    
    return df
    
    
d1=preprocess(df)    

r2_score 0.9134206872323922
rmse 1.9904767217250137
mae 1.5492426388975737
MSE 3.96


C:\Users\ABHIRAM\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- st
Feature names seen at fit time, yet now missing:
- state

  warnings.warn(message, FutureWarning)
C:\Users\ABHIRAM\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- st
Feature names seen at fit time, yet now missing:
- state

  warnings.warn(message, FutureWarning)


In [9]:
# pickle.dump(scaler, open('scaler.pkl', 'wb'))

# pickle.load(open('scaler.pkl','rb'))